In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
churn_df=pd.read_csv("Churn_Modelling.csv")
churn_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## check for missing columns
churn_df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
## check for duplicate entries
churn_df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

In [5]:
churn_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# drop the unnecessary features
churn_df_copy=churn_df

churn_df_copy.drop(labels=["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [7]:
churn_df_copy.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
## Separate the independent and dependent features
X=churn_df_copy.drop(labels=["Exited"],axis=1)
y=churn_df_copy.iloc[:,-1]

In [9]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [10]:
X['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [11]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [12]:
## now apply standard scaling on numerical features and label encoding on categorical features
numerical_features=["CreditScore","Age","Tenure","Balance","EstimatedSalary","NumOfProducts"]
categorical_features=["Geography","Gender"]

In [13]:
# create a categorical transformer and numerical transformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

preprocessor=ColumnTransformer(
    [
        ('oh_encoder',OneHotEncoder(drop='first'),categorical_features),
        ('scaler',StandardScaler(),numerical_features)
    ],
    remainder='passthrough'
)

In [14]:
# train test split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [15]:
# fit_transform on train set and transform on test set
X_train_scaled=preprocessor.fit_transform(X_train)
X_test_scaled=preprocessor.transform(X_test)

In [16]:
preprocessor.get_feature_names_out()

array(['oh_encoder__Geography_Germany', 'oh_encoder__Geography_Spain',
       'oh_encoder__Gender_Male', 'scaler__CreditScore', 'scaler__Age',
       'scaler__Tenure', 'scaler__Balance', 'scaler__EstimatedSalary',
       'scaler__NumOfProducts', 'remainder__HasCrCard',
       'remainder__IsActiveMember'], dtype=object)

In [17]:
# save the preprocessor as pkl file
import pickle

with open('preprocessor.pkl','wb') as file:
    pickle.dump(preprocessor,file)

#### ANN Implementation

In [20]:
import tensorflow as tf
import datetime

In [22]:
# how many no of inputs will go in?
X_train_scaled.shape[1]

11

##### 11 inputs will go to the first hidden layer, so we'll have to mention this during definition of HL1

In [24]:
## Build the ANN model
model=tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation='relu',input_shape=(X_train_scaled.shape[1],)), # HL1 which receives 11 inputs, and has 64 neurons
    tf.keras.layers.Dense(32,activation='relu'), # HL2 with 32 neurons
    tf.keras.layers.Dense(1,activation='sigmoid') # output layer with 1 neuron and sigmoid activation, because binary classification
])

e:\ml-bootcamp\section-53\ANN_Classification\dl_env\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# total no of parameters
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# now we have built a neural network,but we still need to perform forward and backward propagation
# define an optimizer
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [28]:
# now compile the model
# compile-> perform forward and backward propagation
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['recall'])

In [29]:
# set up the tensorboard, because I want to capture my training information once I start training
# set up a log directory
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [30]:
tensorflow_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [35]:
# set up early stopping
early_stopping_callback=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=30,restore_best_weights=True)

In [36]:
# now finally lets train our model
history=model.fit(
    X_train_scaled,y_train,
    validation_data=(X_test_scaled,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.3106 - recall: 0.5201 - val_loss: 0.3382 - val_recall: 0.4728
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3059 - recall: 0.5162 - val_loss: 0.3380 - val_recall: 0.5131
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3021 - recall: 0.5370 - val_loss: 0.3381 - val_recall: 0.4567
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3031 - recall: 0.5195 - val_loss: 0.3393 - val_recall: 0.5131
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2992 - recall: 0.5396 - val_loss: 0.3409 - val_recall: 0.5252
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2991 - recall: 0.5325 - val_loss: 0.3532 - val_recall: 0.4507
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3017 - recall: 0.5344 - val_loss: 0.3490 - val_recall: 0.5131
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.2962 - recall: 0.5455 - val_loss: 0.3613 - val_recall: 0.5835


In [37]:
model.save("model.h5")

In [38]:
# load tensorboard
%load_ext tensorboard